In [1]:
import pandas as pd
import numpy as np
from funciones import contar_valores_sd
from funciones import data_cleaned
import warnings
warnings.filterwarnings("ignore", message="Downcasting.*", category=FutureWarning)

In [2]:
df_lesiones_h = pd.read_excel("DataSets/lesiones.xlsx", sheet_name="HECHOS")

## 3.4. Dataset: Lesiones - Hechos

### 3.3.1. Exploración inicial de los datos

In [3]:
#Vizualizo las primeras filas del DataFrame
df_lesiones_h.head()

,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,gravedad
0,LC-2019-0000179,1,2019,1,1,2019-01-01 00:00:00,09:00:00,9,SD,14,...,-34.559658,CICLISTA,SD,CICLISTA-SD,SD,SD,SD,SD,x,SD
1,LC-2019-0000053,1,2019,1,1,2019-01-01 00:00:00,01:55:00,1,SD,8,...,-34.669125,AUTO,SD,AUTO-SD,SD,x,SD,SD,SD,SD
2,LC-2019-0000063,1,2019,1,1,2019-01-01 00:00:00,02:00:00,2,SD,8,...,-34.677556,SD,SD,SD-SD,SD,SD,SD,SD,SD,SD
3,LC-2019-0000079,1,2019,1,1,2019-01-01 00:00:00,02:30:00,2,SD,7,...,-34.647349,PEATON,SD,PEATON-SD,x,SD,SD,SD,SD,SD
4,LC-2019-0000082,4,2019,1,1,2019-01-01 00:00:00,04:30:00,4,SD,3,...,-34.604579,AUTO,SD,AUTO-SD,SD,SD,x,SD,SD,SD


In [4]:
df_lesiones_h.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     23785 non-null  object 
 1   n_victimas             23785 non-null  int64  
 2   aaaa                   23785 non-null  int64  
 3   mm                     23785 non-null  int64  
 4   dd                     23785 non-null  int64  
 5   fecha                  23785 non-null  object 
 6   hora                   23785 non-null  object 
 7   franja_hora            23780 non-null  object 
 8   direccion_normalizada  23732 non-null  object 
 9   comuna                 23616 non-null  object 
 10  tipo_calle             23785 non-null  object 
 11  otra_direccion         23785 non-null  object 
 12  calle                  12867 non-null  object 
 13  altura                 12771 non-null  float64
 14  cruce                  9407 non-null   object 
 15  ge

### 3.1.2. Limpieza y Normalización de datos

In [5]:
# Usando la funcion contar_valores_sd para ver la cantidad y porcentaje de valores "SD"
contar_sd = contar_valores_sd(df_lesiones_h)
print(contar_sd)

                  Columna  Cantidad de SD  Porcentaje de SD
0                    hora               4          0.016817
1   direccion_normalizada           10815         45.469834
2                  comuna             846          3.556864
3              tipo_calle           11045         46.436830
4          otra_direccion           18295         76.918226
5    geocodificacion_CABA            1213          5.099853
6                longitud            1209          5.083036
7                 latutid            1209          5.083036
8                 victima           10733         45.125079
9                 acusado           15288         64.275804
10                   moto            8511         35.783057
11                   auto           12543         52.734917
12     transporte_publico           11801         49.615304
13                 camion           12708         53.428631
14               ciclista           11353         47.731764
15               gravedad           2305

In [6]:
# Reemplazar "sd" y "SD" con NaN en todo el DataFrame
df_lesiones_h.replace({"sd": np.nan, "SD": np.nan}, inplace=True)

In [7]:
# Contar filas vacías por columna
filas_vacias_por_columna_v = df_lesiones_h.isnull().sum()
print(filas_vacias_por_columna_v)

id                           0
n_victimas                   0
aaaa                         0
mm                           0
dd                           0
fecha                        0
hora                         9
franja_hora                  9
direccion_normalizada    10868
comuna                    1015
tipo_calle               11045
otra_direccion           18295
calle                    10918
altura                   11014
cruce                    14378
geocodificacion_CABA      1252
longitud                  1471
latutid                   1471
victima                  10733
acusado                  15288
participantes                0
moto                      8604
auto                     12636
transporte_publico       11894
camion                   12801
ciclista                 11446
gravedad                 23139
dtype: int64


In [8]:
# Contar filas duplicadas
duplicados_totales = df_lesiones_h.duplicated().sum()
print(duplicados_totales)

0


Como se logra ver nuestro DataFrame no tiene filas duplicadas

In [9]:
# Columnas para eliminar
columns_to_drop = ['aaaa','mm', 'dd', 'hora', 'direccion_normalizada', 'otra_direccion', 'calle', 'altura', 'cruce',
                   'participantes', 'geocodificacion_CABA', 'moto', 'auto', 'transporte_publico', 'camion', 'ciclista']

Decidi eliminar las columnas 'AAAA', 'MM', y 'DD' debido que tenemos una columna 'fecha' donde tenemos todo los datos de las 3 columnas anteriores. Tambien mantiene una estructura más estandarizada, ahorra espacio y facilita el análisis.

Tambien eliminamos la columna 'Hora' debido que la informacion que posee ya la tenemos en otra columna 'HH' y nos facilita el tipo de informacion para trabajar de forma mas rapida

Así mismo las columnas 'LUGAR_DEL_HECHO','Calle','Dirección Normalizada', tienen datos que encontramos en direccion de los hechos y tambien 'XY (CABA)' que contienen coordenadas en formato de proyección cartesiana también fue eliminada. Preferimos trabajar con las columnas de 'latitud' y 'longitud' debido que en el DataFrame anterior venimos trabajando con ese formato

La siguientes columnas 'moto', 'auto', 'transporte_publico', 'camion', 'ciclista' pienso que tienen muchos valores faltantes y complicaria el analis



In [10]:
# Renombrar columnas

rename_dict = {'id': 'id_siniestro', 'n_victimas': 'nro_victimas',
               'latutid': 'latitud', 'victima': 'vehiculo_victima', 'acusado': 'vehiculo_acusado'}

Renombramos columnas para mejorar el entendimiento de la columna(una forma más descrptiva)

In [11]:
# Columnas para pasar a minúcula

columns_to_lower = ['tipo_calle', 'victima', 'acusado', 'gravedad']

Uso la funcion lower para normalizar los valores ,debido que observo los valores en mayuscula y es preferible trabajar todo en minuscula

In [12]:
# Reemplazar valores nulos por otros valores

fill_na_dict = {'gravedad': 'lesiones'}

En la columna 'gravedad' debido que la gran mayoria de sus valores son NaN o nulos decidi reemplazar por un valores común para todos

In [13]:
# Convertir a flotante
columns_to_float = [ 'latitud', 'longitud']

In [14]:
# Convertir a entero
columns_to_int = [ 'nro_victimas', 'franja_hora', 'comuna']

# Categorización de columnas
columns_to_categorize = ['franja_hora', 'tipo_calle', 'comuna', 'vehiculo_victima', 'vehiculo_acusado', 'gravedad']

Convierti las columnas para una mayor facilidad al manipular los datos y tambien la compatibilidad para realizar funciones

In [15]:
# Invocamos la función 'data_cleaning' para que haga el proceso de limpieza de los datos 

df_lesiones_h.cleaned = data_cleaned(df_lesiones_h,
                                    strip_spaces=True, # Eliminar espacios en blanco
                                    drop_columns=columns_to_drop,  # Eliminar columnas
                                    rename_columns=rename_dict,  # Renombrar columnas
                                    lowercase_columns=columns_to_lower, # Convertir a minúsculas 
                                    fill_na=fill_na_dict, # Rellenar nulos                                                             
                                    convert_to_float=columns_to_float, # Convertir a float
                                    convert_to_int_columns=columns_to_int, #Conversión entero
                                    categorize_columns=columns_to_categorize # Categorizar columnas
                                   )

C:\Users\U ANGEL\AppData\Local\Temp\ipykernel_14044\3367288588.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_lesiones_h.cleaned = data_cleaned(df_lesiones_h,


In [16]:
df_lesiones_h.cleaned.head()

,id_siniestro,nro_victimas,fecha,franja_hora,comuna,tipo_calle,longitud,latitud,vehiculo_victima,vehiculo_acusado,gravedad
0,LC-2019-0000179,1,2019-01-01 00:00:00,9,14,NaN,-58.408911,-34.559658,ciclista,NaN,lesiones
1,LC-2019-0000053,1,2019-01-01 00:00:00,1,8,NaN,-58.443510,-34.669125,auto,NaN,lesiones
2,LC-2019-0000063,1,2019-01-01 00:00:00,2,8,NaN,-58.468335,-34.677556,NaN,NaN,lesiones
3,LC-2019-0000079,1,2019-01-01 00:00:00,2,7,NaN,-58.437425,-34.647349,peaton,NaN,lesiones
4,LC-2019-0000082,4,2019-01-01 00:00:00,4,3,NaN,-58.398225,-34.604579,auto,NaN,lesiones


In [17]:
df_lesiones_v = pd.read_excel("DataSets/lesiones.xlsx", sheet_name="VICTIMAS")

## 3.4. Dataset: Lesiones - Víctimas

### 3.4.1. Exploración inicial de los datos

In [18]:
df_lesiones_v.head()

,ID hecho,AAA,MM,DD,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
0,LC-2019-0000053,2019,1,1,2019-01-01,sd,Varon,57,SD
1,LC-2019-0000063,2019,1,1,2019-01-01,sd,SD,SD,SD
2,LC-2019-0000079,2019,1,1,2019-01-01,sd,Varon,SD,SD
3,LC-2019-0000082,2019,1,1,2019-01-01,sd,Varon,45,SD
4,LC-2019-0000082,2019,1,1,2019-01-01,sd,Mujer,45,SD


In [19]:
df_lesiones_v.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID hecho          27605 non-null  object        
 1   AAA               27605 non-null  int64         
 2   MM                27605 non-null  int64         
 3   DD                27605 non-null  int64         
 4   FECHA             27605 non-null  datetime64[ns]
 5   VEHICULO_VICTIMA  27605 non-null  object        
 6   SEXO              27605 non-null  object        
 7   EDAD_VICTIMA      27605 non-null  object        
 8   GRAVEDAD          27605 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 1.9+ MB


### 3.1.2. Limpieza y Normalización de datos

In [20]:
# Usando la funcion contar_valores_sd para ver la cantidad y porcentaje de valores "SD"
contar_sd = contar_valores_sd(df_lesiones_v)
print(contar_sd)

            Columna  Cantidad de SD  Porcentaje de SD
0  VEHICULO_VICTIMA            4479         16.225321
1              SEXO            1929          6.987865
2      EDAD_VICTIMA            3990         14.453903
3          GRAVEDAD           20722         75.066111


In [21]:
# Reemplazar "sd" y "SD" con NaN en todo el DataFrame
df_lesiones_v.replace(["sd", "SD"], np.nan, inplace=True)

In [22]:
# Contar filas vacías por columna
filas_vacias_por_columna_v = df_lesiones_v.isnull().sum()
print(filas_vacias_por_columna_v)

ID hecho                0
AAA                     0
MM                      0
DD                      0
FECHA                   0
VEHICULO_VICTIMA    16272
SEXO                 2228
EDAD_VICTIMA         4471
GRAVEDAD            27068
dtype: int64


In [23]:
# Contar filas duplicadas
duplicados_totales = df_lesiones_v.duplicated().sum()
print(duplicados_totales)

631


Como se logra ver nuestro DataFrame, tienen filas duplicadas pero debido que como los vehiculos no solo llevan un pasajero no borrare filas. Porque muchos de ellos tienen columnas con valores similares pero le distingue un valor sea sexo o edad y no son del todo iguales

In [24]:
# Columnas para eliminar
columns_to_drop = ['FECHA ','AAA', 'MM', 'DD','VEHICULO_VICTIMA','GRAVEDAD']

Las columnas que estamos eliminando 'FECHA','AAAA', 'MM', 'DD','VEHICULO_VICTIMA' y 'GRAVEDAD' ya tenemos en el DataFrame 'Hechos' 

In [25]:
# Renombrar columnas
rename_dict = {'ID hecho': 'id_siniestro', 'SEXO': 'sexo', 'EDAD_VICTIMA': 'edad'}

Renombramos columnas para mejorar el entendimiento de la columna(una forma más descrptiva)

In [26]:
# Columnas para pasar a minúcula

columns_to_lower = ['SEXO']

In [27]:
# Reemplazar valores en columnas
values_to_replace = {'sexo': {'varon': 'masculino', 'mujer': 'femenino'}}

Uso la funcio replace porque es más comun usar 'masculino' y 'femenino' al dirigirse a un hombre y mujer

In [28]:
# Categorización de columnas

columns_to_categorize = ['sexo','edad']

Convierti las columnas para una mayor facilidad al manipular los datos y tambien la compatibilidad para realizar funciones

In [29]:
# Invocamos la función 'data_cleaning' para que haga el proceso de limpieza de los datos 

df_lesiones_v_cleaned = data_cleaned(df_lesiones_v,
                                    strip_spaces=True, # Eliminar espacios en blanco
                                    lowercase_columns=columns_to_lower, # Convertir a minúsculas                                    
                                    drop_columns=columns_to_drop,  # Eliminar columnas
                                    rename_columns=rename_dict,  # Renombrar columnas 
                                    replace_values=values_to_replace, # Reemplazar valores
                                    categorize_columns=columns_to_categorize, # Categorizar columnas
                                    )

c:\Users\U ANGEL\Desktop\Data Sciense\P02_LAB\Lab\funciones.py:120: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df_cleaned[column] = df_cleaned[column].replace(replacements)


In [30]:
df_lesiones_v_cleaned.head()

,id_siniestro,sexo,edad
0,LC-2019-0000053,masculino,57
1,LC-2019-0000063,NaN,NaN
2,LC-2019-0000079,masculino,NaN
3,LC-2019-0000082,masculino,45
4,LC-2019-0000082,femenino,45
